In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from skmap.misc import find_files, GoogleSheet, ttprint

df = pd.read_parquet('./material/pnt_agg.predicted_all.v20250521.gpkg')
geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
df = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:3035")
# de = gpd.read_file("./material/nuts_de_2021.gpkg") 
# es = gpd.read_file("./material/nuts_es_2021.gpkg") 
df = df.rename(columns={'ndvi_glad.landast.ard2.seasconv.m.yearly_p50_30m_s_YYYY0101_YYYY1231_eu_epsg.3035_v20231127':'ndvi'})
df = df[['id', 'nuts0', 'time', 'lon', 'lat', 'pred', 'pred_std', 'ndvi','geometry']]

In [2]:
df_2yrs = df[df['time'].isin([2009, 2018])]

out = (
    df_2yrs
    .pivot_table(
        index=['id', 'nuts0', 'lon', 'lat', 'geometry'],
        columns='time',
        values=['pred', 'pred_std', 'ndvi'],
        aggfunc='first'       
    )
    .dropna(axis=0)           
    .reset_index()
)

out.columns = [
    f'{c[0]}_{c[1]}' if c[1] else c[0]   
    for c in out.columns.to_flat_index()
]

static_cols = ['id', 'nuts0', 'lon', 'lat', 'geometry']
value_cols  = ['pred_2009', 'pred_2018', 'pred_std_2009', 'pred_std_2018', 'ndvi_2009', 'ndvi_2018']
out = out[static_cols + value_cols]

out.head()

,id,nuts0,lon,lat,geometry,pred_2009,pred_2018,pred_std_2009,pred_std_2018,ndvi_2009,ndvi_2018
0,322,DE,4.046952e+06,3.192086e+06,POINT (4046952.408 3192085.582),16.066618,23.510474,7.319217,19.500446,195.0,222.0
1,323,DE,4.046952e+06,3.191086e+06,POINT (4046952.408 3191085.582),24.059321,27.337926,30.668980,25.358030,206.0,217.0
2,324,DE,4.046952e+06,3.190086e+06,POINT (4046952.408 3190085.582),32.421245,34.320420,28.815367,34.057068,221.0,226.0
3,325,DE,4.046952e+06,3.189086e+06,POINT (4046952.408 3189085.582),38.163748,40.007770,34.862160,35.933296,221.0,217.0
4,326,DE,4.046952e+06,3.188086e+06,POINT (4046952.408 3188085.582),41.198004,38.854967,54.757160,49.069435,213.0,216.0


In [3]:
import numpy as np
out['change'] = out['pred_2018'] - out['pred_2009']
out['noise'] = np.sqrt(out['pred_std_2009']**2 + out['pred_std_2018']**2)
out['signal'] = out['change'].abs()
out['SNR'] = out['signal']/out['noise']
out['ndvi_mean'] = (out['ndvi_2009'] + out['ndvi_2018'])/2
out['ndvi_mean'] = (out['ndvi_mean'] - 125)/125

In [5]:
import rasterio
from rasterio.transform import from_origin
from rasterio.features import rasterize

value_col = "SNR"        
points_gdf = out.dropna(subset=[value_col]) 
geometry = [Point(xy) for xy in zip(points_gdf['lon'], points_gdf['lat'])]
points_gdf = gpd.GeoDataFrame(points_gdf, geometry=geometry, crs="EPSG:3035")
# ------------------------------------------------------------------
# 2.  DERIVE GRID RESOLUTION & EXTENT ------------------------------
# ------------------------------------------------------------------

x_res = 1000    # ≈ pixel width  (e.g. 1000 m)
y_res = 1000           # ≈ pixel height (e.g. 1000 m)

minx, miny, maxx, maxy = points_gdf.total_bounds

# number of pixels in each direction (+1 so last column/row is included)
width  = int(round((maxx - minx) / x_res)) + 1
height = int(round((maxy - miny) / y_res)) + 1

# Affine transform (upper-left corner is half a pixel north-west of first point)
transform = from_origin(minx - x_res / 2,   # west-most edge
                        maxy + y_res / 2,   # north-most edge
                        x_res,
                        y_res)

# ------------------------------------------------------------------
# 3.  RASTERIZE POINT VALUES ---------------------------------------
# ------------------------------------------------------------------
shapes = ((geom, val) for geom, val in
          zip(points_gdf.geometry, points_gdf[value_col]))

raster = rasterize(
    shapes=shapes,
    out_shape=(height, width),
    transform=transform,
    fill=np.nan,             # background
    dtype="float32"          # change if your data need another type
)

# ------------------------------------------------------------------
# 4.  WRITE GeoTIFF -----------------------------------------------
# ------------------------------------------------------------------
meta = {
    "driver":  "GTiff",
    "height":  height,
    "width":   width,
    "count":   1,
    "dtype":   "float32",
    "crs":     points_gdf.crs,   # keep original CRS
    "transform": transform,
    "nodata":  np.nan
}

with rasterio.open(f"./figure/{value_col}_new.tif", "w", **meta) as dst:
    dst.write(raster, 1)

print("GeoTIFF written to predictions.tif")


GeoTIFF written to predictions.tif
